### Python金融风控策略实战（基于Jupyter Notebook）
## <center>德国信贷风控数据建模（步骤3：特征工程之变量分箱）</center>
### <center>策略制定及验证：张君颖  ； 报告日期：2020.11.11</center>
  <font color=blue><center>本报告不构成投资建议，转载需注明作者且不得删改</center></font>
  <font color=blue><center>作者邮箱：zhang.jun.ying@outlook.com</center></font>

### 第一步：导入需要使用的python库，并进行数据描述

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore") # 忽略警告

### 数据概况
数据来源：加州大学（UCI）欧文机器学习库    
http://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29  

数据概况：“德国信用数据”将申请贷款的人分为“好信用”和“坏信用”，样本数据1000人，数据维度20维   

特征向量：1.支票账户状态；2.借款周期；3.历史信用；4.借款目的；5.信用额度；6.储蓄账户状态；7.当前就业状态；8.分期付款占可支配收入百分比；9.性别与婚姻状态；10.他人担保信息；11.现居住地；12.财产状态；13.年龄；14.其他分期情况；15.房产状态；16.信用卡数量；17.工作状态；18.赡养人数；19.电话号码注册情况；20.是否有海外工作经历

特征向量对应英文：1.status_account, 2.duration, 3.credit_history, 4,purpose, 5.amount, 6.svaing_account, 7.present_emp, 8.income_rate, 9.personal_status, 10.other_debtors, 11.residence_info, 12.property, 13.age, 14.inst_plans, 15.housing, 16.num_credits, 17.job, 18.dependents, 19.telephone, 20.foreign_worker    

输出结果：好信用（0），坏信用（1）   

数据收录时间：1994-11-17    

数据上传者：Professor Dr. Hans Hofmann 汉斯霍夫曼博士（德国汉堡大学）  

### 第二步：下载数据至本地，保存成csv格式，使用pandas导入数据
将数据分为data_train（训练集）和 data_test（测试集）两部分，其中训练集占总数据量80%，即800条样本数据，测试集占20%，即200条样本数据，两个数据集均通过随机抽取得到。   

In [2]:
def data_read(data_path,file_name):
    df = pd.read_csv( os.path.join(data_path, file_name), delim_whitespace = True, header = None )
    columns = ['status_account','duration','credit_history','purpose', 'amount',
               'svaing_account', 'present_emp', 'income_rate', 'personal_status',
               'other_debtors', 'residence_info', 'property', 'age',
               'inst_plans', 'housing', 'num_credits',
               'job', 'dependents', 'telephone', 'foreign_worker', 'target']
    df.columns = columns
    # 将标签变量由状态1,2转为0,1; 其中0表示好信用，1表示坏信用
    df.target = df.target - 1
    # 将数据分为data_train（训练集）和 data_test（测试集）两部分
    data_train, data_test = train_test_split(df, test_size=0.2, random_state=0,stratify=df.target)
    # 由于训练集、测试集是随机划分，索引是乱的，需要重新排序
    data_train = data_train.reset_index(drop=True)
    data_test = data_test.reset_index(drop=True)
    return data_train, data_test

if __name__ == '__main__':
    data_path = os.path.join('C:\\Users\\lotbear\\Desktop\\Data-science！','金融风控实战数据')
    file_name = 'german.csv'
    # 读取数据
    data_train, data_test = data_read(data_path,file_name)

### 第三步：定义变量分箱函数
常用的四种分箱方法：   

<font color=red>1: Chi-merge (卡方分箱)</font>     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 如果卡方值较小，则相邻两个区间的类分布情况相似，可以进行合并；反之，卡方值越大，则相邻区间的类分布情况不同，不能进行区间合并。     
   
<font color=red>2: Information Value (最优IV值分箱) </font>       
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a. IV值的计算是在WOE值的基础上演化而来（关于WOE编码，请见该系列的《步骤2：对离散数据进行编码》）;    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b. 在切分点初分裂得到的两部分数据中，选择好坏样本分布差异最大点作为最优切分点；   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c. 变量的IV值越大，对目标变量的区分能力越强。

<font color=red>3: 信息熵 (基于树的分箱)</font>   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a. 决策树采用自顶向下递归的方法进行树的生成，每个节点的选择目标是为了分类结果的纯度更高，因此，不同的损失函数有不同的决策树；    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b. 本例采用信息熵增益作为目标函数进行变量分箱，信息熵是不确定性度量的平均值，与纯度反向对应，熵越大，纯度越低，反之亦然；   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c. 分类的类别越少，纯度越高，只有一个类别的数据，纯度最高；    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; d. 如果在某个切点进行分箱，使得信息熵下降，则得到最佳分箱切分点。  

<font color=red>4: Best-KS 分箱法</font>     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; a. 将可排序的离散变量/连续变量进行升序排列后，确定初始阀点候选集（将离散变量的可能取值设为初始阀点；将连续变量先分组，再将每组边界作为初始阀点值）；   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; b. 分别计算阀点值区间内，好坏样本与总样本中好坏样本的比值，将两个比值做差，得到各个切分点的KS统计值；   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; c. Best-KS 只能对二分类问题进行分箱，且不适用于不可排序的离散变量，因此该方法不在代码中演示。

### 连续变量分箱函数 ---> cont_var_bin()   

<font color=red>continue-variable-bins</font>

在定义 cont_var_bin() 函数前，先定义其需要调用的7个过程子函数：
1. init_equal_bin() 初始化等距分组函数
2. cont_var_bin_map()分箱映射函数
3. limit_min_sample() 分箱最小样本数约束函数
4. select_split_point() 二叉树分割点函数
5. merge_bin() 区间合并函数
6. best_split() 候选集中寻找切分点的函数
7. cal_advantage() 切分点指标函数

其中，函数 6 和 7 是函数 4 的子函数，因此需要先于函数 4 进行定义

=========================================================  

子函数1 >>> init_equal_bin() 初始化等距分组函数：   
    
参数   
<font color=red>x</font>:要分组的变量值，pandas series   
<font color=red>bin_rate</font>：比例值1/bin_rate   

返回值   
返回初始化分箱结果：pandas dataframe 

In [3]:
#  cont_var_bin() 的中间过程函数（子函数<1>）
def init_equal_bin(x,bin_rate):
   
    # 异常值剔除，只考虑95%设的最大值与最小值，边界与-inf或inf分为一组
    if len(x[x > np.percentile(x, 95)]) > 0 and len(np.unique(x)) >=30:
        var_up= min( x[x > np.percentile(x, 95)] )
    else:
        var_up = max(x)
    if len(x[x < np.percentile(x, 5)]) > 0:
        var_low= max( x[x < np.percentile(x, 5)] )
    else:
        var_low = min(x)
        
    # 初始化分组
    bin_num = int(1/ bin_rate)
    dist_bin = (var_up - var_low) / bin_num  # 分箱间隔
    bin_up = []
    bin_low = []
    for i in range(1, bin_num + 1):
        if i == 1:
            bin_up.append( var_low + i * dist_bin)
            bin_low.append(-np.inf)
        elif i == bin_num:
            bin_up.append( np.inf)
            bin_low.append( var_low + (i - 1) * dist_bin )
        else:
            bin_up.append( var_low + i * dist_bin )
            bin_low.append( var_low + (i - 1) * dist_bin )
    result = pd.DataFrame({'bin_up':bin_up,'bin_low':bin_low})
    result.index.name = 'bin_num'
    return result

子函数2 >>> cont_var_bin_map()   

分箱映射函数：按照初始化分箱结果，对原始值进行分箱映射，用于训练集与测试集的分箱映射 

In [4]:
# cont_var_bin() 的中间过程函数（子函数<2>）
def cont_var_bin_map(x, bin_init):
    
    temp = x.copy()
    for i in bin_init.index:
        bin_up = bin_init['bin_up'][i]
        bin_low = bin_init['bin_low'][i]
        # 寻找出 >lower and <= upper的位置
        if pd.isnull(bin_up) or pd.isnull(bin_up):
            temp[pd.isnull(temp)] = i
        else:
            index = (x > bin_low) & (x <= bin_up)
            temp[index] = i
    temp.name = temp.name + "_BIN"
    return temp

子函数3 >>> limit_min_sample()    

分箱最小样本数约束函数：每个箱内的样本数不能小于bin_min_num_0。   

参数   
<font color=red>temp_cont</font> : 初始化分箱后的结果 pandas dataframe     
<font color=red>bin_min_num_0</font>：每组内的最小样本限制     

返回值   
合并后的分箱结果：pandas dataframe  

In [5]:
# cont_var_bin() 的中间过程函数（子函数<3>）
def limit_min_sample(temp_cont,  bin_min_num_0):

    for i in temp_cont.index:
        rowdata = temp_cont.loc[i, :]
        if i == temp_cont.index.max():
            # 如果是最后一个箱就，取倒数第二个值
            ix = temp_cont[temp_cont.index < i].index.max()
        else:
            # 否则就取大于i的最小的分箱值
            ix = temp_cont[temp_cont.index > i].index.min()
        # 如果0, 1, total项中样本的数量小于20则进行合并
        if rowdata['total'] <= bin_min_num_0:
            # 与相邻的bin合并
            temp_cont.loc[ix, 'bad'] = temp_cont.loc[ix, 'bad'] + rowdata['bad']
            temp_cont.loc[ix, 'good'] = temp_cont.loc[ix, 'good'] + rowdata['good']
            temp_cont.loc[ix, 'total'] = temp_cont.loc[ix, 'total'] + rowdata['total']
            if i < temp_cont.index.max():
                temp_cont.loc[ix, 'bin_low'] = rowdata['bin_low']
            else:
                temp_cont.loc[ix, 'bin_up'] = rowdata['bin_up']
            temp_cont = temp_cont.drop(i, axis=0)  
    return temp_cont.sort_values(by='bad_rate')

子函数6 >>> best_split()    

候选集中寻找切分点的函数：在每个候选集中寻找切分点，完成一次分裂。  

参数    
<font color=red>df_temp0</font> : 上一步的分箱结果，pandas dataframe    
<font color=red>bin_num</font> : 分箱编号，在不同编号的分箱结果中继续二分    
<font color=red>method</font> : 分箱方法选择，1: chi-merge , 2: IV值, 3: 信息熵   

返回值   
返回在本次分箱标号内的最有切分结果，pandas dataframe  

In [6]:
def best_split(df_temp0, method, bin_num):

#    df_temp0 = df_temp
#    bin_num = 1
    df_temp0 = df_temp0.sort_values(by=['bin', 'bad_rate'])
    piont_len = len(df_temp0[df_temp0['bin'] == bin_num])  #候选集的长度
    bestValue = 0
    bestI = 1
    # 以候选集的每个切分点做分隔，计算指标值
    for i in range(1, piont_len):
        #计算指标值 
        value = cal_advantage(df_temp0,i,method,flag='sel')
        if bestValue < value:
            bestValue = value
            bestI = i
    # create new var split
    df_temp0['split'] = np.where(df_temp0['bin_raw'] <= bestI, 1, 0)
    df_temp0 = df_temp0.drop('bin_raw', axis=1)
    newbinDS = df_temp0.sort_values(by=['split', 'bad_rate'])
    # rebuild var i
    newbinDS_0 = newbinDS[newbinDS['split'] == 0]
    newbinDS_1 = newbinDS[newbinDS['split'] == 1]
    newbinDS_0 = newbinDS_0.copy()
    newbinDS_1 = newbinDS_1.copy()
    newbinDS_0['bin_raw'] = range(1, len(newbinDS_0) + 1)
    newbinDS_1['bin_raw'] = range(1, len(newbinDS_1) + 1)
    newbinDS = pd.concat([newbinDS_0, newbinDS_1], axis=0)
    return newbinDS  

子函数7 >>> cal_advantage()   

切分点指标函数：计算当前切分点下的指标值   

参数    
<font color=red>temp</font> : 上一步的分箱结果，pandas dataframe    
<font color=red>piont</font> : 切分点，以此来划分分箱    
<font color=red>method</font> : 分箱方法选择，1: chi-merge , 2: IV值, 3: 信息熵     

In [7]:
def cal_advantage(temp, piont, method,flag='sel'):

#    temp = binDS
    if flag == 'sel':
        # 用于最优切分点选择，这里只是二叉树，即二分
        bin_num = 2
        good_bad_matrix = np.empty((bin_num, 3))
        for ii in range(bin_num):
            if ii==0:
                df_temp_1 = temp[temp['bin_raw'] <= piont]
            else:
                df_temp_1 = temp[temp['bin_raw'] > piont]
            # 计算每个箱内的好坏样本书
            good_bad_matrix[ii][0] = df_temp_1['good'].sum()
            good_bad_matrix[ii][1] = df_temp_1['bad'].sum()
            good_bad_matrix[ii][2] = df_temp_1['total'].sum()
                    
    
    elif flag == 'gain':
       # 用于计算本次分箱后的指标结果，即分箱数，每增加一个，就要算一下当前分箱下的指标结果
       bin_num = temp['bin'].max()
       good_bad_matrix = np.empty((bin_num, 3))
       for ii in range(bin_num):
           df_temp_1 = temp[temp['bin'] == (ii + 1)]
           good_bad_matrix[ii][0] = df_temp_1['good'].sum()
           good_bad_matrix[ii][1] = df_temp_1['bad'].sum()
           good_bad_matrix[ii][2] = df_temp_1['total'].sum()
       
    # 计算总样本中的好坏样本
    total_matrix = np.empty(3)
    total_matrix[0] = temp.good.sum()
    total_matrix[1] = temp.bad.sum()
    total_matrix[2] = temp.total.sum()
    
    # Chi-merger分箱
    if method == 1:
        X2 = 0
        for i in range(bin_num):
            for j in range(2):
                expect = (total_matrix[j] / total_matrix[2])*good_bad_matrix[i][2]
                X2 = X2 + (good_bad_matrix[i][j] - expect )**2/expect
        M_value = X2
    
    # IV分箱
    elif method == 2:
        if pd.isnull(total_matrix[0]) or  pd.isnull(total_matrix[1]) or total_matrix[0] == 0 or total_matrix[1] == 0:
            M_value = np.NaN
        else:
            IV = 0
            for i in range(bin_num):
                # 坏好比
                weight = good_bad_matrix[i][1] / total_matrix[1] - good_bad_matrix[i][0] / total_matrix[0]
                IV = IV + weight * np.log( (good_bad_matrix[i][1] * total_matrix[0]) / (good_bad_matrix[i][0] * total_matrix[1]))
            M_value = IV
    
    # 信息熵分箱
    elif method == 3:
        # 总的信息熵    
        entropy_total = 0
        for j in range(2):
            weight = (total_matrix[j]/ total_matrix[2])
            entropy_total = entropy_total - weight * (np.log(weight))
                    
        # 计算条件熵
        entropy_cond = 0
        for i in range(bin_num):
            entropy_temp = 0
            for j in range(2):
                entropy_temp = entropy_temp - ((good_bad_matrix[i][j] / good_bad_matrix[i][2]) \
                                         * np.log(good_bad_matrix[i][j] / good_bad_matrix[i][2]) )
            entropy_cond = entropy_cond + good_bad_matrix[i][2]/total_matrix[2] * entropy_temp 
        
        # 计算归一化信息增益   
        M_value = 1 - (entropy_cond / entropy_total)  
        
    # Best-Ks分箱
    else:
        pass
    return M_value

子函数4 >>> select_split_point()   

二叉树分割点函数：从候选者中挑选每次的最优切分点，与切分后的指标计算。  

参数   
<font color=red>temp_bin</font>: 分箱后的结果 pandas dataframe   
<font color=red>method</font>:分箱方法选择，1: chi-merge , 2: IV值, 3: 信息熵    

返回值   
新的分箱结果 : pandas dataframe

In [8]:
# cont_var_bin() 的中间过程函数（子函数<4>）
def select_split_point(temp_bin, method):
    
#    temp_bin = df_temp_all
    temp_bin = temp_bin.sort_values(by=['bin', 'bad_rate'])
    # 得到最大的分箱值
    max_num = max(temp_bin['bin'])
#    temp_binC = dict()
#    m = dict()
#    ##不同箱内的数据取出来
#    for i in range(1, max_num + 1):
#        temp_binC[i] = temp_bin[temp_bin['bin'] == i]
#        m[i] = len(temp_binC[i])
    temp_main = dict()
    bin_i_value = []
    for i in range(1, max_num + 1):
        df_temp = temp_bin[temp_bin['bin'] == i]
        if df_temp.shape[0]>1 : 
            # bin=i的做分裂
            temp_split= best_split(df_temp, method, i) # 调用子函数(6)
            # 完成一次分箱，更新bin的节点
            temp_split['bin'] = np.where(temp_split['split'] == 1,
                                               max_num + 1,
                                               temp_split['bin'])
            # 取出bin!=i合并为新组
            temp_main[i] = temp_bin[temp_bin['bin'] != i]
            temp_main[i] = pd.concat([temp_main[i], temp_split ], axis=0, sort=False)
            # 计算新分组的指标值
            value = cal_advantage(temp_main[i],0, method,flag='gain') # 调用子函数(7)
            newdata = [i, value]
            bin_i_value.append(newdata)
    # find maxinum of value bintoSplit
    bin_i_value.sort(key=lambda x: x[1], reverse=True)
    # binNum = temp_all_Vals['BinToSplit']
    binNum = bin_i_value[0][0]
    newBins = temp_main[binNum].drop('split', axis=1)
    return newBins.sort_values(by=['bin', 'bad_rate']), round( bin_i_value[0][1] ,4)

子函数5 >>> merge_bin()   

区间合并函数：将相同箱内的样本书合并，区间合并。  

参数   
<font color=red> sub </font> : 分箱结果子集，pandas dataframe ，如 bin=1 的结果   
<font color=red> i </font> : 分箱标号    

返回值    
返回合并结果    

In [9]:
def merge_bin(sub, i):

    l = len(sub)
    total = sub['total'].sum()
    first = sub.iloc[0, :]
    last = sub.iloc[l - 1, :]

    lower = first['bin_low']
    upper = last['bin_up']
    df = pd.DataFrame()
    df = df.append([i, lower, upper, total], ignore_index=True).T
    df.columns = ['bin', 'bin_low', 'bin_up', 'total']
    return df

### 将7个过程子函数封装进 cont_var_bin() 连续变量分箱函数

<font color=red>x</font> :输入分箱数据，pandas series；    
<font color=red>y</font> :标签变量；    
<font color=red>method</font> :分箱方法选择，1: chi-merge , 2: IV值, 3: 基尼系数分箱；    
<font color=red>mmin</font> :最小分箱数，当分箱初始化后如果初始化箱数小于等 mmin，则 mmin=2，即最少分2箱，如果分两箱也无法满足箱内最小样本数限制而分1箱，则变量删除；   
<font color=red>mmax</font> :最大分箱数，当分箱初始化后如果初始化箱数小于等于 mmax，则 mmax 等于初始化箱数-1；   
<font color=red>bin_rate</font> ：等距初始化分箱参数，分箱数为1/bin_rate,分箱间隔在数据中的最小值与最大值将等间隔取值；   
<font color=red>stop_limit</font> :分箱 earlystopping 机制，如果已经没有明显增益即停止分箱；   
<font color=red>bin_min_num</font> :每组最小样本数。    

返回值   
分箱结果：<font color=red> pandas dataframe </font> 

In [10]:
# 连续变量分箱函数
def cont_var_bin(x, y, method, mmin=5, mmax=10, bin_rate=0.01, stop_limit=0.1, bin_min_num=20):

    # 缺失值单独取出来
    df_na = pd.DataFrame({'x': x[pd.isnull(x)], 'y': y[pd.isnull(x)]})
    y = y[~pd.isnull(x)]
    x = x[~pd.isnull(x)]
    # print('将缺失值单独取出后，输入变量的数值统计如下：')
    # print(x)
    # print('='*80)
  
    # 初始化分箱，等距的方式，后面加上约束条件,没有箱内样本数没有限制
    bin_init = init_equal_bin(x, bin_rate) # 调用子函数（1）
    # 分箱映射
    bin_map = cont_var_bin_map(x, bin_init) # 调用子函数（2）
    
    df_temp = pd.concat([x, y, bin_map], axis=1)
    # 计算每个bin中好坏样本的频数
    df_temp_1 = pd.crosstab(index=df_temp[bin_map.name], columns=y)
    df_temp_1.rename(columns= dict(zip([0,1], ['good', 'bad'])) , inplace=True)
    # 计算每个bin中一共有多少样本
    df_temp_2 = pd.DataFrame(df_temp.groupby(bin_map.name).count().iloc[:, 0])
    df_temp_2.columns = ['total']
    df_temp_all= pd.merge(pd.concat([df_temp_1, df_temp_2], axis=1), bin_init,
                         left_index=True, right_index=True,
                         how='left')
    print('初始化分箱后，每个箱中样本数量:')
    print(df_temp_all)
    print('='*80)
    
    # 做分箱上下限的整理，让候选点连续
    for j in range(df_temp_all.shape[0]-1):
        if df_temp_all.bin_low.loc[df_temp_all.index[j+1]] !=  df_temp_all.bin_up.loc[df_temp_all.index[j]]:
            df_temp_all.bin_low.loc[df_temp_all.index[j+1]] = df_temp_all.bin_up.loc[df_temp_all.index[j]]
        
    # 离散变量中这个值为badrate,连续变量时为索引，索引值是分箱初始化时，箱内有变量的箱的索引
    df_temp_all['bad_rate'] = df_temp_all.index
    
    # 最小样本数限制，进行分箱合并
    df_temp_all = limit_min_sample(df_temp_all, bin_min_num) # 调用子函数（3）
    
    # 将合并后的最大箱数与设定的箱数进行比较，这个应该是分箱数的最大值
    if mmax >= df_temp_all.shape[0]:
        mmax = df_temp_all.shape[0]-1
    if mmin >= df_temp_all.shape[0]:
        gain_value_save0=0
        gain_rate_save0=0
        df_temp_all['bin'] = np.linspace(1,df_temp_all.shape[0],df_temp_all.shape[0],dtype=int)
        data = df_temp_all[['bin_low','bin_up','total','bin']]
        data.index = data['bin']
        
    else:
        df_temp_all['bin'] = 1
        df_temp_all['bin_raw'] = range(1, len(df_temp_all) + 1)
        df_temp_all['var'] = df_temp_all.index  # 初始化箱的编号
        gain_1 = 1e-10
        gain_rate_save0 = []
        gain_value_save0 = []
        
        # 分箱约束：最大分箱数限制
        for i in range(1,mmax):
    #        i = 1
            df_temp_all, gain_2 = select_split_point(df_temp_all, method=method) # 调用子函数（4）
            gain_rate = gain_2 / gain_1 - 1  # ratio gain
            gain_value_save0.append(np.round(gain_2,4))
            if i == 1:
                gain_rate_save0.append(0.5)
            else:
                gain_rate_save0.append(np.round(gain_rate,4))
            gain_1 = gain_2
            if df_temp_all.bin.max() >= mmin and df_temp_all.bin.max() <= mmax:
                if gain_rate <= stop_limit or pd.isnull(gain_rate):
                    break
                
    
        df_temp_all = df_temp_all.rename(columns={'var': 'oldbin'})
        temp_Map1 = df_temp_all.drop(['good', 'bad', 'bad_rate', 'bin_raw'], axis=1)
        temp_Map1 = temp_Map1.sort_values(by=['bin', 'oldbin'])
        # print('按约束条件重新进行分箱，得到分箱结果：（4<bins<10）')
        # print(temp_Map1)
        # print('='*80)
        
        # get new lower, upper, bin, total for sub
        data = pd.DataFrame()
        for i in temp_Map1['bin'].unique():
            # 得到这个箱内的上下界
            sub_Map = temp_Map1[temp_Map1['bin'] == i]
            rowdata = merge_bin(sub_Map, i) # 调用子函数（5）
            data = data.append(rowdata, ignore_index=True)
        # print('新的分箱约束条件下，各个分箱的样本数据量：')
        # print(data)
        # print('='*80)
    
        # resort data
        data = data.sort_values(by='bin_low')
        data = data.drop('bin', axis=1)
        mmax = df_temp_all.bin.max()
        data['bin'] = range(1, mmax + 1)
        data.index = data['bin']
        # print('将新分箱结果进行排序：')
        # print(data)
        # print('='*80)
        
    # 将缺失值的箱加过来
    if len(df_na) > 0:
        row_num = data.shape[0] + 1
        data.loc[row_num, 'bin_low'] = np.nan
        data.loc[row_num, 'bin_up'] = np.nan
        data.loc[row_num, 'total'] = df_na.shape[0]
        data.loc[row_num, 'bin'] = data.bin.max() + 1
        
    print('按约束条件重新进行分箱，最终得到分箱结果：（按bin_low排序）')
    print(data)
    print('='*80)
    print('gain_value_save0:','\n',gain_value_save0)
    print('gain_rate_save0:','\n',gain_rate_save0)
    return data , gain_value_save0 ,gain_rate_save0

### 离散变量分箱函数 ---> disc_var_bin()   

<font color=red> discrete-variable-bins</font>

在定义 cont_var_bin() 函数前，先定义其需要调用的1个过程子函数：  
8. cal_bin_value() 按变量类别分箱函数

===========================================================    
子函数8 >>> cal_bin_value()   

按变量类别分箱函数：离散变量分箱初始化，不满足最小样本数的箱进行合并。  

参数   
<font color=red> x </font> : 待分箱的离散变量 pandas Series   
<font color=red> y </font> : 标签变量    
<font color=red> target </font> : 正样本标识
    
返回值：计算结果  

In [11]:
def cal_bin_value(x, y, bin_min_num_0=10):

    # 按类别 x 计算 y 中 0,1 两种状态的样本数
    df_temp = pd.crosstab(index=x, columns=y, margins=False)
    df_temp.rename(columns= dict(zip([0,1], ['good', 'bad'])) , inplace=True)
    
    # 增加 4 列统计数据：total、bin、var_name、bad_rate
    df_temp = df_temp.assign(total=lambda x:x['good']+ x['bad'],bin=1,
                             var_name=df_temp.index).assign(bad_rate=lambda x:x['bad']/ x['total'])
    # print('按变量的类别，初步统计 good 和 bad 信用的样本数：')
    # print(df_temp)
    # print('='*80)

    # 按照 bad_rate 排序
    df_temp = df_temp.sort_values(by='bad_rate')
    df_temp = df_temp.reset_index(drop=True)
    print('初始化分箱，以bad-rate排序，统计不同类别下 good 和 bad 信用分布：')
    print(df_temp)
    print('='*80)
    
    # 样本数不满足最小值，则合并箱
    for i in df_temp.index:
        rowdata = df_temp.loc[i, :]
        if i == df_temp.index.max():
            # 如果是最后一个箱，就取倒数第二个值
            ix = df_temp[df_temp.index < i].index.max()
        else:
            # 否则就取大于i的最小的分箱值
            ix = df_temp[df_temp.index > i].index.min()
        # 如果0, 1, total项中样本的数量小于20则进行合并
        if any(rowdata[:3] <= bin_min_num_0):
            # 与相邻的bin合并
            df_temp.loc[ix, 'bad'] = df_temp.loc[ix, 'bad'] + rowdata['bad']
            df_temp.loc[ix, 'good'] = df_temp.loc[ix, 'good'] + rowdata['good']
            df_temp.loc[ix, 'total'] = df_temp.loc[ix, 'total'] + rowdata['total']
            df_temp.loc[ix, 'bad_rate'] = df_temp.loc[ix,'bad'] / df_temp.loc[ix, 'total']
            # 将区间也进行合并
            df_temp.loc[ix, 'var_name'] = str(rowdata['var_name']) +'%'+ str(df_temp.loc[ix, 'var_name'])
         
            df_temp = df_temp.drop(i, axis=0)  # 删除原来的bin
    # 如果离散变量小于等于5，每个变量为一个箱
    df_temp['bin_raw'] = range(1, df_temp.shape[0] + 1)
    df_temp = df_temp.reset_index(drop=True)
    print('对初始分箱结果进行约束后，不同类别下 good 和 bad 信用分布：')
    print(df_temp)
    print('='*80)
    return df_temp

### 将子函数封装进 disc_var_bin() 离散变量分箱函数
离散变量分箱方法，如果变量过于稀疏最好先编码，再按连续变量进行分箱。   

参数：   
<font color=red>x</font> : 输入分箱数据，pandas series；   
<font color=red>y</font> : 标签变量；   
<font color=red>method</font> : 分箱方法选择，1:chi-merge , 2:IV值, 3:信息熵；  
<font color=red>mmin</font> : 最小分箱数，当分箱初始化后如果初始化箱数小于等mmin，则mmin=2，即最少分2箱，如果分两厢也无法满足箱内最小样本数限制而分1箱，则变量删除；  
<font color=red>mmax</font> : 最大分箱数，当分箱初始化后如果初始化箱数小于等于mmax，则mmax等于初始化箱数-1；     
<font color=red>stop_limit</font> : 分箱earlystopping机制，如果已经没有明显增益即停止分箱；      
<font color=red>bin_min_num</font> : 每组最小样本数。  

返回值   
分箱结果 ：<font color=red>pandas dataframe</font>   

In [12]:
def disc_var_bin(x, y, method=1, mmin=3, mmax=8, stop_limit=0.1, bin_min_num = 20  ):
    
#    x = data_train.purpose
#    y = data_train.target
    del_key = []    
    # 缺失值单独取出来
    df_na = pd.DataFrame({'x': x[pd.isnull(x)], 'y': y[pd.isnull(x)]})
    y = y[~pd.isnull(x)]
    x = x[~pd.isnull(x)]
    # 数据类型转化
    if np.issubdtype(x.dtype, np.int_):
        x = x.astype('float').astype('str')
    if np.issubdtype(x.dtype, np.float_):
        x = x.astype('str')
  
    # 按照类别分箱，得到每个箱下的统计值
    temp_cont = cal_bin_value(x, y,bin_min_num) # 调用子函数（8）
    
    # 如果去掉缺失值后离散变量的可能取值小于等于5不分箱
    if len(x.unique()) > 5:
        # 将合并后的最大箱数与设定的箱数进行比较，这个应该是分箱数的最大值
        if mmax >= temp_cont.shape[0]:
            mmax = temp_cont.shape[0]-1
        if mmin >= temp_cont.shape[0]:
            mmin = 2
            mmax = temp_cont.shape[0]-1
        if mmax ==1:
            print('变量 {0}合并后分箱数为1，该变量删除'.format(x.name))
            del_key.append(x.name)
        
        gain_1 = 1e-10
        gain_value_save0 = []
        gain_rate_save0 = []
        for i in range(1,mmax):
            temp_cont, gain_2 = select_split_point(temp_cont, method=method) # 调用子函数（4）
            gain_rate = gain_2 / gain_1 - 1  # ratio gain
            gain_value_save0.append(np.round(gain_2,4))
            if i == 1:
                gain_rate_save0.append(0.5)
            else:
                gain_rate_save0.append(np.round(gain_rate,4))
            gain_1 = gain_2
            if temp_cont.bin.max() >= mmin and temp_cont.bin.max() <= mmax:
                if gain_rate <= stop_limit:
                    break
        print('gain_value_save0:',gain_value_save0)
        print('='*80)
        print('gain_rate_save0:',gain_rate_save0)
        print('='*80)
    
        temp_cont = temp_cont.rename(columns={'var': x.name})
        temp_cont = temp_cont.drop(['good', 'bad', 'bin_raw', 'bad_rate'], axis=1)
    else:
        temp_cont.bin = temp_cont.bin_raw
        temp_cont = temp_cont[['total', 'bin', 'var_name']]
        gain_value_save0=[]
        gain_rate_save0=[]
        del_key=[]
    # 将缺失值的箱加过来
    if len(df_na) > 0:
        index_1 = temp_cont.shape[0] + 1
        temp_cont.loc[index_1, 'total'] = df_na.shape[0]
        temp_cont.loc[index_1, 'bin'] = temp_cont.bin.max() + 1
        temp_cont.loc[index_1, 'var_name'] = 'NA'
    temp_cont = temp_cont.reset_index(drop=True)

    if temp_cont.shape[0]==1:
        del_key.append(x.name)
        
    print('最终分箱结果：(含缺失数据箱)')
    print(temp_cont.sort_values(by='bin'))
    print('='*80)
    
    return temp_cont.sort_values(by='bin') , gain_value_save0 , gain_rate_save0,del_key

disc_var_bin_map()   
离散变量分箱映射函数：用离散变量分箱后的结果，对原始值进行分箱映射。  

参数   
<font color=red>x</font> : 待分箱映射的离散变量，pandas Series   
<font color=red>bin_map</font> : 分箱映射字典， pandas dataframe   
 
返回映射结果   

In [13]:
def disc_var_bin_map(x, bin_map):

    # 数据类型转化
    xx = x[~pd.isnull(x)]
    if np.issubdtype(xx.dtype, np.int_):
        x[~pd.isnull(x)] = xx.astype('float').astype('str')
    if np.issubdtype(xx.dtype, np.float_):
        x[~pd.isnull(x)] = xx.astype('str') 
    d = dict()
    for i in bin_map.index:
        for j in  bin_map.loc[i,'var_name'].split('%'):
            if j != 'NA':
                d[j] = bin_map.loc[i,'bin']

    new_x = x.map(d)
    # 有缺失值要做映射
    if sum(pd.isnull(new_x)) > 0:
        index_1 = bin_map.index[bin_map.var_name == 'NA']
        if len(index_1) > 0:
            new_x[pd.isnull(new_x)] = bin_map.loc[index_1,'bin'].tolist()
    new_x.name = x.name + '_BIN'

    return new_x

### 第四步：调用函数，对连续变量进行分箱
举例：针对一个连续型变量 amount 信用额度 ，采用三种分箱方法，分别进行分箱。  

训练数据 data_train 共计800条，含29条缺失值数据。

In [14]:
print('@以连续变量【 amount 信用额度】举例，用三种分箱方法分别进行分箱')
print(' '*80)
# 人为制造29条缺失数据
data_train.amount[1:30] = np.nan
print('~'*80)
print('###基于Chi-merge (卡方分箱法)进行分箱：###')
print(' '*80)
data_test1,gain_value_save1 ,gain_rate_save1  = cont_var_bin(data_train.amount, data_train.target, 
                            method=1, mmin=4 ,mmax=10,bin_rate=0.01,stop_limit=0.1 ,bin_min_num=20 )
print('~'*80)
print('###基于Information Value (最优IV值分箱法)进行分箱：###') 
print(' '*80)
data_test2,gain_value_save2 ,gain_rate_save2  = cont_var_bin(data_train.amount, data_train.target,
                            method=2, mmin=4 ,mmax=10,bin_rate=0.01,stop_limit=0.1 ,bin_min_num=20 )
print('~'*80)
print('###基于信息熵 (树的分箱法)进行分箱：###')
print(' '*80)
data_test3,gain_value_save3 ,gain_rate_save3 = cont_var_bin(data_train.amount, data_train.target, 
                            method=3, mmin=4 ,mmax=10,bin_rate=0.01,stop_limit=0.1 ,bin_min_num=20 )

@以连续变量【 amount 信用额度】举例，用三种分箱方法分别进行分箱
                                                                                
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
###基于Chi-merge (卡方分箱法)进行分箱：###
                                                                                
初始化分箱后，每个箱中样本数量:
            good  bad  total   bin_up  bin_low
amount_BIN                                    
0.0           30   19     49   786.78     -inf
1.0            6    3      9   864.56   786.78
2.0           10    6     16   942.34   864.56
3.0            6    4     10  1020.12   942.34
4.0           11    3     14  1097.90  1020.12
...          ...  ...    ...      ...      ...
95.0           1    0      1  8175.88  8098.10
96.0           0    1      1  8253.66  8175.88
97.0           0    1      1  8331.44  8253.66
98.0           1    2      3  8409.22  8331.44
99.0          18   22     40      inf  8409.22

[97 rows x 5 columns]
按约束条件重新进行分箱，最终得到分箱结果：（按bin_low排序）
    

In [15]:
# 对连续变量进行批量分箱，把每个变量分箱的结果保存在字典中
print('对7个连续变量进行批量分箱:')
dict_cont_bin = {}
cont_name = ['duration', 'amount', 'income_rate',  'residence_info',  
            'age',  'num_credits','dependents']

for i in cont_name:
    print('~'*80)
    print('连续变量名:',i)
    dict_cont_bin[i],gain_value_save , gain_rate_save = cont_var_bin(data_train[i], 
                                      data_train.target, method=1, mmin=4, mmax=10,
                                      bin_rate=0.01, stop_limit=0.1, bin_min_num=20)

对7个连续变量进行批量分箱:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
连续变量名: duration
初始化分箱后，每个箱中样本数量:
              good  bad  total  bin_up  bin_low
duration_BIN                                   
0                7    0      7    5.49     -inf
2               51    7     58    6.47     5.98
4                2    0      2    7.45     6.96
6                4    0      4    8.43     7.94
8               29    8     37    9.41     8.92
10              20    3     23   10.39     9.90
12               5    0      5   11.37    10.88
14             100   36    136   12.35    11.86
16               4    0      4   13.33    12.84
18               2    1      3   14.31    13.82
20              42   11     53   15.29    14.80
22               1    1      2   16.27    15.78
26              55   37     92   18.23    17.74
30               7    1      8   20.19    19.70
32              19    8     27   21.17    20.68
38             105   48    153   24.11    23.62
42     

初始化分箱后，每个箱中样本数量:
                good  bad  total  bin_up  bin_low
dependents_BIN                                   
0                466  202    668    1.01     -inf
99                94   38    132     inf     1.99
按约束条件重新进行分箱，最终得到分箱结果：（按bin_low排序）
     bin_low  bin_up  total  bin
bin                             
1       -inf    1.01    668    1
2       1.01     inf    132    2
gain_value_save0: 
 0
gain_rate_save0: 
 0


### 第五步：调用函数，对离散变量进行分箱

In [16]:
print('@以离散变量【 purpose 借款目的】举例，用三种分箱方法分别进行分箱')
print(' '*80)

# 人为制造29条缺失数据
data_train.purpose[1:30] = np.nan
print('~'*80)
print('###基于Chi-merge (卡方分箱法)进行分箱：###')
print(' '*80)
data_disc_test1,gain_value_save1 ,gain_rate_save1,del_key  = disc_var_bin(data_train.purpose, data_train.target, 
                            method=1, mmin=4 ,mmax=10,stop_limit=0.1 ,bin_min_num=10 )
print('~'*80)
print('###基于Information Value (最优IV值分箱法)进行分箱：###') 
print(' '*80)
data_disc_test2,gain_value_save2 ,gain_rate_save2 ,del_key = disc_var_bin(data_train.purpose, data_train.target,
                            method=2, mmin=4 ,mmax=10,stop_limit=0.1 ,bin_min_num=10 )
print('~'*80)
print('###基于信息熵 (树的分箱法)进行分箱：###')
print(' '*80)
data_disc_test3,gain_value_save3 ,gain_rate_save3,del_key = disc_var_bin(data_train.purpose, data_train.target, 
                            method=3, mmin=4 ,mmax=10,stop_limit=0.1 ,bin_min_num=10 )

@以离散变量【 purpose 借款目的】举例，用三种分箱方法分别进行分箱
                                                                                
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
###基于Chi-merge (卡方分箱法)进行分箱：###
                                                                                
初始化分箱，以bad-rate排序，统计不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate
0          7    1      8    1      A48  0.125000
1         67   10     77    1      A41  0.129870
2        171   42    213    1      A43  0.197183
3         91   40    131    1      A42  0.305344
4        115   72    187    1      A40  0.385027
5         50   32     82    1      A49  0.390244
6          6    4     10    1      A44  0.400000
7          8    6     14    1      A45  0.428571
8         20   19     39    1      A46  0.487179
9          5    5     10    1     A410  0.500000
对初始分箱结果进行约束后，不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin          var_name  bad_rate  bin_raw
0  

In [17]:
# 对离散变量进行批量分箱，把每个变量分箱的结果保存在字典中
print('对13个离散变量进行批量分箱:')
dict_disc_bin = {}
del_key = []
disc_name = [x for x in data_train.columns if x not in cont_name]
disc_name.remove('target')
for i in disc_name:
    print('~'*80)
    print('离散变量名：',i)
    dict_disc_bin[i],gain_value_save , gain_rate_save,del_key_1  = disc_var_bin(data_train[i], 
                                      data_train.target, method=1, mmin=3,
                                      mmax=8, stop_limit=0.1, bin_min_num=5)
    if len(del_key_1)>0 :
        del_key.extend(del_key_1)
        
print('='*80)
print('分箱后，无法满足约束条件的变量为：',del_key)

对13个离散变量进行批量分箱:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
离散变量名： status_account
初始化分箱，以bad-rate排序，统计不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate
0        284   39    323    1      A14  0.120743
1         40   11     51    1      A13  0.215686
2        124   84    208    1      A12  0.403846
3        112  106    218    1      A11  0.486239
对初始分箱结果进行约束后，不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate  bin_raw
0        284   39    323    1      A14  0.120743        1
1         40   11     51    1      A13  0.215686        2
2        124   84    208    1      A12  0.403846        3
3        112  106    218    1      A11  0.486239        4
最终分箱结果：(含缺失数据箱)
target  total  bin var_name
0         323    1      A14
1          51    2      A13
2         208    3      A12
3         218    4      A11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
离散变量名： credit_history
初始化分箱，以b

初始化分箱，以bad-rate排序，统计不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate
0        115   43    158    1     A172  0.272152
1        353  151    504    1     A173  0.299603
2         12    6     18    1     A171  0.333333
3         80   40    120    1     A174  0.333333
对初始分箱结果进行约束后，不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate  bin_raw
0        115   43    158    1     A172  0.272152        1
1        353  151    504    1     A173  0.299603        2
2         12    6     18    1     A171  0.333333        3
3         80   40    120    1     A174  0.333333        4
最终分箱结果：(含缺失数据箱)
target  total  bin var_name
0         158    1     A172
1         504    2     A173
2          18    3     A171
3         120    4     A174
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
离散变量名： telephone
初始化分箱，以bad-rate排序，统计不同类别下 good 和 bad 信用分布：
target  good  bad  total  bin var_name  bad_rate
0        238   89    327    1     A192  

In [18]:
if len(del_key) > 0:
    print('删除分箱数只有1个的变量:',del_key)
    for j in del_key:
        del dict_disc_bin[j]

删除分箱数只有1个的变量: ['foreign_worker']


### 第五步：训练数据集的连续 / 离散变量分箱映射
针对训练集800个样本数据，根据变量的不同类别进行分箱。   

In [19]:
# print('训练数据集的连续变量分箱映射:7个连续变量')
#    ss = data_train[list( dict_cont_bin.keys())]
df_cont_bin_train = pd.DataFrame()
for i in dict_cont_bin.keys():
    df_cont_bin_train = pd.concat([ df_cont_bin_train , 
                                   cont_var_bin_map(data_train[i], dict_cont_bin[i]) ], axis = 1)
# print(df_cont_bin_train)

In [20]:
# print('训练数据集的离散变量分箱映射:12个离散变量')
#    ss = data_train[list( dict_disc_bin.keys())]
df_disc_bin_train = pd.DataFrame()
for i in dict_disc_bin.keys():
    df_disc_bin_train = pd.concat([ df_disc_bin_train , 
                                   disc_var_bin_map(data_train[i], dict_disc_bin[i]) ], axis = 1)
# print(df_disc_bin_train)

### 第六步：测试数据集的连续 / 离散变量分箱映射
针对测试集200个样本数据，根据变量的不同类别进行分箱，打印出样本分箱的一一对应关系。   

In [21]:
print('测试数据集的连续变量分箱映射:7个连续变量')
ss = data_test[list( dict_cont_bin.keys())]
df_cont_bin_test = pd.DataFrame()
for i in dict_cont_bin.keys():
    df_cont_bin_test = pd.concat([ df_cont_bin_test , 
                                  cont_var_bin_map(data_test[i], dict_cont_bin[i]) ], axis = 1)
print(df_cont_bin_test)

测试数据集的连续变量分箱映射:7个连续变量
     duration_BIN  amount_BIN  income_rate_BIN  residence_info_BIN  age_BIN  \
0               1           1                3                   4        5   
1               4           2                4                   2        1   
2               2           2                2                   3        4   
3               5           3                4                   3        3   
4               5           4                4                   4        5   
..            ...         ...              ...                 ...      ...   
195             2           4                2                   2        3   
196             3           4                2                   2        3   
197             5           5                2                   3        5   
198             4           2                2                   4        4   
199             4           1                4                   2        3   

     num_credits_BIN  depende

In [22]:
print('训练数据集的离散变量分箱映射:12个离散变量')
#    ss = data_test[list( dict_disc_bin.keys())]
df_disc_bin_test = pd.DataFrame()
for i in dict_disc_bin.keys():
    df_disc_bin_test = pd.concat([ df_disc_bin_test , 
                                  disc_var_bin_map(data_test[i], dict_disc_bin[i]) ], axis = 1)
print(df_disc_bin_test)

训练数据集的离散变量分箱映射:12个离散变量
     status_account_BIN  credit_history_BIN  purpose_BIN  svaing_account_BIN  \
0                     4                   2          4.0                   4   
1                     3                   2          4.0                   4   
2                     4                   2          4.0                   4   
3                     4                   2          2.0                   4   
4                     3                   2          1.0                   4   
..                  ...                 ...          ...                 ...   
195                   3                   2          4.0                   4   
196                   3                   2          4.0                   3   
197                   4                   2          1.0                   4   
198                   2                   1          4.0                   1   
199                   1                   2          3.0                   2   

     present_emp

### 篇外彩蛋：关于 DataFrame 数据类型，提取非缺失/缺失数据

In [23]:
a=pd.DataFrame(np.array([[1,2,3],[4,np.nan,6],[7,8,9]]),columns=['a', 'b', 'c'])
a

,a,b,c
0,1.0,2.0,3.0
1,4.0,NaN,6.0
2,7.0,8.0,9.0


打印含缺失值的行

In [24]:
a[a.isnull().values==True]

,a,b,c
1,4.0,NaN,6.0


判断数据是否为空

In [25]:
pd.isnull(a)

,a,b,c
0,False,False,False
1,False,True,False
2,False,False,False


In [26]:
~5

-6

In [27]:
# 提取非缺失数据
b=a[~pd.isnull(a)]
b

,a,b,c
0,1.0,2.0,3.0
1,4.0,NaN,6.0
2,7.0,8.0,9.0


In [28]:
# 提取缺失数据
c=a[pd.isnull(a)]
c

,a,b,c
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
